In [1]:
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

In [3]:
dataset_name = 'Flowers102'

dataset = torchvision.datasets.Flowers102(
    root='./data',             # Directory to save or load data
    # transform=transform,       # Apply the defined transformations
    download=True              # Download the dataset if not already downloaded
)

100%|██████████| 344862509/344862509 [04:06<00:00, 1398549.56it/s]


Extracting data\flowers-102\102flowers.tgz to data\flowers-102


100%|██████████| 502/502 [00:00<00:00, 35952.20it/s]


100%|██████████| 14989/14989 [00:00<00:00, 9686968.05it/s]
